In [12]:
import pandas as pd
from itertools import chain
from lnn import Predicate, Variable, Exists, Implies, And, ForAll, Model, Fact, World

![Hierarchy](https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Quadrilateral_hierarchy_svg.svg/629px-Quadrilateral_hierarchy_svg.svg.png)

In [13]:
entities = pd.read_csv("geometry_entities.csv", index_col=0)
entities.head(5)

,3-sides-equal trapezoid,bicentric,complex,concave,convex,cyclic,isosceles trapezoid,kite,parallelogram,quadrilateral,rectangle,rhombus,right-angled trapezoid,simple,square,tangential,trapezoid
3-sides-equal trapezoid,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
bicentric,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
complex,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
concave,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
convex,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1


In [14]:
properties = pd.read_csv("geometry_properties.csv", index_col=0)

properties.head(5)

,3-sides-equal trapezoid,bicentric,complex,concave,convex,cyclic,isosceles trapezoid,kite,parallelogram,quadrilateral,rectangle,rhombus,right-angled trapezoid,simple,square,tangential,trapezoid
foursided,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
not-self-intersecting,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
self-intersecting,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no-reflex-interior-angle,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
atleast-one-reflex-interior-angle,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
def parse_model(entities: pd.DataFrame, properties: pd.DataFrame) -> Model:
    """Parse model from entities and properties matrix.

    Args:
        entities (pd.DataFrame): bin matrix which represents mutual relationship between entities (logical implication)
        properties (pd.DataFrame): bin matrix which represents relationship between properties and entities (logical implication)

    Returns:
        Model: parsed model
    """

    def _parse_implications(model, table):
        x = Variable("x")

        return [
            ForAll(
                x,
                Implies(model[item1](x), model[item2](x), name=f"{item1}_{item2}"),
                name=f"all-{item1}_{item2}",
                world=World.AXIOM,
            )
            for item1, row in table.items()
            for item2, value in row.items()
            if value
        ]

    model = Model()

    # parsing predicates
    for predicate in chain(entities.index, properties.index):
        model[predicate] = Predicate(name=predicate)

    # parsing entity relations
    model.add_formulae(*_parse_implications(model, entities))

    # parsing entity properties
    model.add_formulae(*_parse_implications(model, properties))

    return model


In [16]:
model = parse_model(entities, properties)

In [17]:
model.print()


***************************************************************************
                                LNN Model

AXIOM  ForAll: all-trapezoid_atleast-one-pair-opposite-sides-parallel (x)          TRUE (1.0, 1.0)

OPEN   Implies: trapezoid_atleast-one-pair-opposite-sides-parallel(x) 

AXIOM  ForAll: all-tangential_sides-are-tangents-to-inscribed-circle (x)          TRUE (1.0, 1.0)

OPEN   Implies: tangential_sides-are-tangents-to-inscribed-circle(x) 

AXIOM  ForAll: all-simple_not-self-intersecting (x)          TRUE (1.0, 1.0)

OPEN   Implies: simple_not-self-intersecting(x) 

AXIOM  ForAll: all-right-angled trapezoid_two-adjacent-right-angles (x)          TRUE (1.0, 1.0)

OPEN   Implies: right-angled trapezoid_two-adjacent-right-angles(x) 

AXIOM  ForAll: all-rhombus_all-sides-equal-length (x)          TRUE (1.0, 1.0)

OPEN   Implies: rhombus_all-sides-equal-length(x) 

AXIOM  ForAll: all-rectangle_all-right-angles (x)           TRUE (1.0, 1.0)

OPEN   Implies: rectangle_all-rig

In [18]:
# gegnerate one object of all entity types
facts = dict([(entity, [f"{entity}1"]) for entity in entities.index])
facts

{'3-sides-equal trapezoid': ['3-sides-equal trapezoid1'],
 'bicentric': ['bicentric1'],
 'complex': ['complex1'],
 'concave': ['concave1'],
 'convex': ['convex1'],
 'cyclic': ['cyclic1'],
 'isosceles trapezoid': ['isosceles trapezoid1'],
 'kite': ['kite1'],
 'parallelogram': ['parallelogram1'],
 'quadrilateral': ['quadrilateral1'],
 'rectangle': ['rectangle1'],
 'rhombus': ['rhombus1'],
 'right-angled trapezoid': ['right-angled trapezoid1'],
 'simple': ['simple1'],
 'square': ['square1'],
 'tangential': ['tangential1'],
 'trapezoid': ['trapezoid1']}

In [19]:
model.add_facts(
    {entity: {name: Fact.TRUE for name in names} for entity, names in facts.items()}
)

In [20]:
x = Variable("x")

queries = [
    Exists(x, model["foursided"](x), name="foursided_objects"),
    Exists(
        x,
        And(model["all-right-angles"](x), model["all-sides-equal-length"](x)),
        name="all-right-angles-and-all-sides-equal-length",
    )
]

model.add_formulae(*queries)

In [21]:
steps, facts_inferred = model.infer()

for query in queries:
    print(f"Query {query.name} result:")
    print(model[query.name].true_groundings)

Query foursided_objects result:
{'complex1', 'quadrilateral1'}
Query all-right-angles-and-all-sides-equal-length result:
{'square1'}


In [22]:
model.print()


***************************************************************************
                                LNN Model

OPEN   Exists: all-right-angles-and-all-sides-equal-length (x)          TRUE (1.0, 1.0)

OPEN   And: And_1(x) 
'right-angled trapezoid1'                                UNKNOWN (0.0, 1.0)
'isosceles trapezoid1'                                   UNKNOWN (0.0, 1.0)
'trapezoid1'                                             UNKNOWN (0.0, 1.0)
'bicentric1'                                             UNKNOWN (0.0, 1.0)
'parallelogram1'                                         UNKNOWN (0.0, 1.0)
'kite1'                                                  UNKNOWN (0.0, 1.0)
'tangential1'                                            UNKNOWN (0.0, 1.0)
'square1'                                                   TRUE (1.0, 1.0)
'cyclic1'                                                UNKNOWN (0.0, 1.0)
'3-sides-equal trapezoid1'                               UNKNOWN (0.0, 1.0)
'rectangl